***/* PART 2: SQLite
/* We now want you to jump over to a local instance of the database on your machine. 

Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it. 

Make sure that the SQLFiles folder containing thes files is in your working directory, and
that you haven't changed the name of the .db file from 'sqlite\db\pythonsqlite'.

You should see the output from the initial query 'SELECT * FROM FACILITIES'.

Complete the remaining tasks in the Jupyter interface. If you struggle, feel free to go back
to the PHPMyAdmin interface as and when you need to. 

You'll need to paste your query into value of the 'query1' variable and run the code block again to get an output.
 
QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


/* Q12: Find the facilities with their usage by member, but not guests */


/* Q13: Find the facilities usage by month, but not guests */;***

In [187]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [189]:
import sqlite3
import pandas as pd

In [191]:
database = database = '/Users/shaymaremy/Library/CloudStorage/OneDrive-Personal/Desktop/Springboard 2024 Data Science/unit 19-SQL & databases/SQL case study/SQLFiles Tier 1/sqlite_db_pythonsqlite.db'

In [193]:
con = sqlite3.connect(database)

In [195]:
query = "SELECT * FROM Facilities;"

In [197]:
df = pd.read_sql_query(query, con)

In [199]:
print(df)

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   
5      5   Massage Room 2         9.9       80.0           4000   
6      6     Squash Court         3.5       17.5           5000   
7      7    Snooker Table         0.0        5.0            450   
8      8       Pool Table         0.0        5.0            400   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  
5                3000  
6                  80  
7                  15  
8                  15  


In [202]:
#/* Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members! */
query_q10 = '''
SELECT f.name AS facility_name, 
       SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END) AS total_revenue
FROM Facilities f
JOIN Bookings b ON f.facid = b.facid
GROUP BY f.name
HAVING total_revenue < 1000
ORDER BY total_revenue;
'''

In [204]:
df_q10 = pd.read_sql_query(query_q10, con)
print("Facilities with total revenue less than 1000:")
print(df_q10)

Facilities with total revenue less than 1000:
   facility_name  total_revenue
0   Table Tennis            180
1  Snooker Table            240
2     Pool Table            270


In [206]:
#/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */
query_q11 = '''
SELECT m.firstname, m.surname, m.recommendedby
FROM Members m
ORDER BY m.surname, m.firstname;
'''

In [208]:
df_q11 = pd.read_sql_query(query_q11, con)
print("\nMembers and who recommended them:")
print(df_q11)


Members and who recommended them:
    firstname            surname recommendedby
0    Florence              Bader             9
1        Anne              Baker             9
2     Timothy              Baker            13
3         Tim             Boothe             3
4      Gerald            Butters             1
5        Joan             Coplin            16
6       Erica            Crumpet             2
7       Nancy               Dare             4
8       David            Farrell              
9      Jemima            Farrell              
10      GUEST              GUEST              
11    Matthew            Genting             5
12       John               Hunt            30
13      David              Jones             4
14    Douglas              Jones            11
15     Janice           Joplette             1
16       Anna          Mackenzie             1
17    Charles               Owen             1
18      David             Pinker            13
19  Millicent            

In [210]:
#/* Q12: Find the facilities with their usage by member, but not guests */
query_q12 = '''
SELECT f.name AS facility_name,
        COUNT(b.memid) AS usage_count
FROM Facilities f
JOIN Bookings b ON f.facid = b.facid
WHERE b.memid != 0 -- Exclude guests
GROUP BY f.name;
'''

In [212]:
df_q12 = pd.read_sql_query(query_q12, con)
print("\nFacilities usage by member:")
print(df_q12)


Facilities usage by member:
     facility_name  usage_count
0  Badminton Court          344
1   Massage Room 1          421
2   Massage Room 2           27
3       Pool Table          783
4    Snooker Table          421
5     Squash Court          195
6     Table Tennis          385
7   Tennis Court 1          308
8   Tennis Court 2          276


In [214]:
#/* Q13: Find the facilities usage by month, but not guests */;

query_q13 = '''
SELECT f.name AS facility_name, 
       strftime('%Y-%m', b.starttime) AS month, 
       COUNT(b.memid) AS usage_count
FROM Facilities f
JOIN Bookings b ON f.facid = b.facid
WHERE b.memid != 0  -- Exclude guests
GROUP BY f.name, month
ORDER BY month;
'''

In [216]:
df_q13 = pd.read_sql_query(query_q13, con)
print("\nFacilities usage by month:")
print(df_q13)


Facilities usage by month:
      facility_name    month  usage_count
0   Badminton Court  2012-07           51
1    Massage Room 1  2012-07           77
2    Massage Room 2  2012-07            4
3        Pool Table  2012-07          103
4     Snooker Table  2012-07           68
5      Squash Court  2012-07           23
6      Table Tennis  2012-07           48
7    Tennis Court 1  2012-07           65
8    Tennis Court 2  2012-07           41
9   Badminton Court  2012-08          132
10   Massage Room 1  2012-08          153
11   Massage Room 2  2012-08            9
12       Pool Table  2012-08          272
13    Snooker Table  2012-08          154
14     Squash Court  2012-08           85
15     Table Tennis  2012-08          143
16   Tennis Court 1  2012-08          111
17   Tennis Court 2  2012-08          109
18  Badminton Court  2012-09          161
19   Massage Room 1  2012-09          191
20   Massage Room 2  2012-09           14
21       Pool Table  2012-09          408
22    